In [19]:
import pandas as pd
import numpy as np
from utils import utils
import functools as ft

In [20]:
df_papers = pd.read_excel('papers_processed\papers_final.xlsx')
df_data = pd.read_csv('combined data\yearly_jrc_agri4cast_and_soil_data.csv')
df_papers.head()

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\y'
<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\y'
C:\Users\artur\AppData\Local\Temp\ipykernel_21572\1163103810.py:1: SyntaxWarning: invalid escape sequence '\p'
  df_papers = pd.read_excel('papers_processed\papers_final.xlsx')
C:\Users\artur\AppData\Local\Temp\ipykernel_21572\1163103810.py:2: SyntaxWarning: invalid escape sequence '\y'
  df_data = pd.read_csv('combined data\yearly_jrc_agri4cast_and_soil_data.csv')


,doi,selection type,selection,selection information page,farming practice,result,result information page,crop type,carbon footprint,carbon footprint information page,results_detailed,results actual
0,10.1016/bs.agron.2017.11.005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10.1007/s11104-018-3619-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.1088/1748-9326/aa67bd,latitude,<35,NaN,biochar amendment,Crop yield +25% (median biochar rate 15 t/ha),2026-02-03 00:00:00,Various crops (meta-analysis of 109 studies),NaN,NaN,yield,"""+25"""
3,10.1088/1748-9326/aa67bd,latitude,>35,NaN,biochar amendment,Crop yield -3% (median biochar rate 30 t/ha),2026-02-03 00:00:00,Various crops (meta-analysis of 109 studies),NaN,NaN,yield,"""-3"""
4,10.1007/s11056-015-9491-7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_data.head()

,Unnamed: 0,lon,lat,yearly_rain_6,yearly_rain_7,yearly_rain_8,yearly_rain_9,yearly_rain_10,yearly_rain_11,yearly_rain_12,...,nitrogen,sand,clay,silt,cfvo,soc,oc,ocd_mean,bdod_mean,cec_mean
0,0,31.938307,26.862311,0.2,0.0,2.2,0.1,17.200000,1.5,1.0,...,66.5657,481.9009,305.9815,212.1203,263.2598,41.9028,18.2268,118.4097,133.3838,158.8610
1,1,32.255928,26.862311,0.2,0.0,2.5,0.1,17.900000,1.4,1.2,...,67.8604,479.1478,286.4404,234.4126,263.1713,45.5470,18.4873,110.5106,134.2556,166.0894
2,2,32.573549,26.862311,0.2,0.0,2.7,0.0,19.100000,0.8,1.1,...,73.3602,466.6892,275.3424,257.9714,325.3555,58.4900,17.9992,119.6809,134.8348,158.5099
3,3,32.891170,26.862311,0.2,0.0,3.0,0.0,21.600000,0.6,1.1,...,107.3417,535.1691,244.3798,220.4520,248.5240,45.1325,17.6230,120.5966,132.6689,151.6080
4,4,33.208791,26.862311,0.1,0.0,3.9,0.0,24.800001,0.4,1.2,...,89.3602,435.4464,250.0902,314.4668,351.7962,74.0498,15.5543,132.6581,135.7268,166.4694


In [22]:
df_data['ph'] = df_data['ph']/10
df_data['ph']

0        8.107330
1        8.091160
2        7.987220
3        8.036680
4        7.905980
           ...   
18156    5.926995
18157    5.883975
18158    5.961862
18159    6.013064
18160    6.036842
Name: ph, Length: 18161, dtype: float64

In [23]:
def process_numerical(env_data, row, selection_env, temp):
    print(row['selection'])
    selection = row['selection']
    if "-" in selection:
        bounds = selection.split('-')
        # print(bounds)
        satisfying_rows = env_data[(env_data[selection_env] > float(bounds[0]))]
        satisfying_rows = satisfying_rows[(satisfying_rows[selection_env] < float(bounds[1]))]
        satisfying_rows['results_detailed'], satisfying_rows['results actual'], satisfying_rows['farming practice'] = row['results_detailed'], row['results actual'], row['farming practice']
        temp.append(satisfying_rows)
        # print(satisfying_rows)
    elif ">" in selection:
        bound = selection[1:]
        satisfying_rows = env_data[(env_data[selection_env] > float(bound))]
        satisfying_rows['results_detailed'], satisfying_rows['results actual'], satisfying_rows['farming practice'] = row['results_detailed'], row['results actual'], row['farming practice']
        temp.append(satisfying_rows)
    elif "<" in selection:
        bound = selection[1:]
        satisfying_rows = env_data[(env_data[selection_env] < float(bound))]
        satisfying_rows['results_detailed'], satisfying_rows['results actual'], satisfying_rows['farming practice'] = row['results_detailed'], row['results actual'], row['farming practice']
        temp.append(satisfying_rows)
    return temp

In [24]:
process_dict = {
    'yearly precipitation': 'precip_mean',
    'latitude': 'lat',
    'soil ph': 'ph',
    'yearly temperature': 'mean_temp_mean'
}

In [25]:
def combine(env_data, paper_data):
    temp = []
    for idx, row in paper_data.iterrows():
        sel_type = row['selection type']
        if sel_type in process_dict.keys():
            print(process_dict[sel_type])
            temp = process_numerical(env_data, row, process_dict[sel_type], temp)
    combined = pd.concat(temp, ignore_index=True)   
    return combined
            
                

In [26]:
c = combine(df_data, df_papers)
c

lat
<35
lat
>35
ph
<7 
ph
>7 
precip_mean
<200 
precip_mean
200-400
precip_mean
400-800
precip_mean
>800
mean_temp_mean
>12
mean_temp_mean
<12
precip_mean
<200
precip_mean
200-400
precip_mean
400-800
precip_mean
>800
mean_temp_mean
>9
mean_temp_mean
<9
mean_temp_mean
>9
ph
<7
ph
>7
ph
<7
ph
>7
ph
< 7
ph
>7


C:\Users\artur\AppData\Local\Temp\ipykernel_21572\1545004792.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  satisfying_rows['results_detailed'], satisfying_rows['results actual'], satisfying_rows['farming practice'] = row['results_detailed'], row['results actual'], row['farming practice']
C:\Users\artur\AppData\Local\Temp\ipykernel_21572\1545004792.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  satisfying_rows['results_detailed'], satisfying_rows['results actual'], satisfying_rows['farming pract

,Unnamed: 0,lon,lat,yearly_rain_6,yearly_rain_7,yearly_rain_8,yearly_rain_9,yearly_rain_10,yearly_rain_11,yearly_rain_12,...,silt,cfvo,soc,oc,ocd_mean,bdod_mean,cec_mean,results_detailed,results actual,farming practice
0,0,31.938307,26.862311,0.20000,0.00000,2.20000,0.10000,17.200000,1.50000,1.00000,...,212.12030,263.259800,41.9028,18.226800,118.40970,133.383800,158.86100,yield,"""+25""",biochar amendment
1,1,32.255928,26.862311,0.20000,0.00000,2.50000,0.10000,17.900000,1.40000,1.20000,...,234.41260,263.171300,45.5470,18.487300,110.51060,134.255600,166.08940,yield,"""+25""",biochar amendment
2,2,32.573549,26.862311,0.20000,0.00000,2.70000,0.00000,19.100000,0.80000,1.10000,...,257.97140,325.355500,58.4900,17.999200,119.68090,134.834800,158.50990,yield,"""+25""",biochar amendment
3,3,32.891170,26.862311,0.20000,0.00000,3.00000,0.00000,21.600000,0.60000,1.10000,...,220.45200,248.524000,45.1325,17.623000,120.59660,132.668900,151.60800,yield,"""+25""",biochar amendment
4,4,33.208791,26.862311,0.10000,0.00000,3.90000,0.00000,24.800001,0.40000,1.20000,...,314.46680,351.796200,74.0498,15.554300,132.65810,135.726800,166.46940,yield,"""+25""",biochar amendment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172622,10306,53.218914,52.907233,336.10016,402.40012,333.70007,279.70020,286.000060,370.10020,290.50003,...,363.74466,116.678680,654.4815,71.215320,486.36246,105.395294,349.74423,"yield,wue","""+15,+15""",aerated irrigation (AI)
172623,10308,53.854156,52.907233,317.50020,394.20004,326.20004,268.40012,279.000030,327.10007,275.10013,...,342.50030,114.554540,642.0357,70.496895,485.00570,105.189255,350.66592,"yield,wue","""+15,+15""",aerated irrigation (AI)
172624,10501,50.995567,53.224854,366.40027,402.50000,336.00015,310.70007,306.000030,509.40005,362.00018,...,479.00513,109.621080,633.6929,65.017900,390.24597,105.932820,313.36212,"yield,wue","""+15,+15""",aerated irrigation (AI)
172625,10502,51.313188,53.224854,355.90024,402.20016,326.00015,298.40005,294.199950,483.10004,360.40018,...,454.08432,107.950226,622.1509,67.849030,415.71823,104.763350,324.44556,"yield,wue","""+15,+15""",aerated irrigation (AI)


In [27]:
c.to_csv('example.csv', index=False)